<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import folium

# Загрузка данных
df_trips = pd.read_csv("trips.csv", parse_dates=["Дата_Время"])
df_trips = df_trips.sort_values(["№_машины", "Дата_Время"])  # Сортируем по времени

gdf_bus_lanes = gpd.read_file("bus_lanes.geojson")  # Загружаем выделенные полосы
gdf_bus_lanes = gdf_bus_lanes.set_crs("EPSG:4326")  # Указываем систему координат

# Создаем траектории машин
def create_trajectories(df):
    trajs = []
    for car_id, group in df.groupby("№_машины"):
        points = [Point(lon, lat) for lon, lat in zip(group["Долгота"], group["Широта"])]
        trajs.append({"№_машины": car_id, "geometry": LineString(points)})
    return gpd.GeoDataFrame(trajs, crs="EPSG:4326")

gdf_trips = create_trajectories(df_trips)

# Определяем пересечения с выделенными полосами
gdf_trips["on_bus_lane"] = gdf_trips.intersects(gdf_bus_lanes.unary_union)

# Анализ скорости на участках
df_trips["next_time"] = df_trips.groupby("№_машины")["Дата_Время"].shift(-1)
df_trips["next_lon"] = df_trips.groupby("№_машины")["Долгота"].shift(-1)
df_trips["next_lat"] = df_trips.groupby("№_машины")["Широта"].shift(-1)

# Вычисление времени и расстояний
from geopy.distance import geodesic

def compute_speed(row):
    if pd.isnull(row["next_time"]):
        return None
    distance = geodesic((row["Широта"], row["Долгота"]), (row["next_lat"], row["next_lon"])).meters
    time_diff = (row["next_time"] - row["Дата_Время"]).total_seconds()
    return distance / time_diff if time_diff > 0 else None

df_trips["speed"] = df_trips.apply(compute_speed, axis=1)

# Средняя скорость на выделенных полосах и вне их
avg_speed_bus_lane = 25  # Можно получить из данных общественного транспорта
avg_speed_normal = df_trips["speed"].median()

# Оценка выигрыша во времени
df_trips["predicted_time"] = df_trips.apply(
    lambda row: row["distance"] / avg_speed_bus_lane if row["on_bus_lane"] else row["distance"] / row["speed"],
    axis=1
)
df_trips["time_saved"] = df_trips["time"] - df_trips["predicted_time"]

# Итоговый выигрыш во времени
total_time_saved = df_trips["time_saved"].sum()

print(f"Общая экономия времени: {total_time_saved / 3600:.2f} часов")
